In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import skimage.io as io

from PIL import Image

import matplotlib.pyplot as plt

import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from skimage.segmentation import watershed
from skimage.feature import peak_local_max

import cv2
import pyvista as pv

from pathlib import Path
import zipfile

import sklearn.feature_extraction
import sklearn.feature_extraction
from skimage.segmentation import expand_labels, watershed
import skimage.morphology

from sklearn.cluster import spectral_clustering

from skimage import morphology


import deepdrr
from deepdrr import Volume
from deepdrr import geo
from deepdrr.utils import test_utils

import nibabel as nib

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# vol_path = Path("/home/pelvisvr/PrephixData/ct_datasets/PrephixNMDID/nifti/BONE_H-N-UXT_3X3/case-100065/segmentations/pulmonary_vein.nii.gz")
vol_path = Path("/home/pelvisvr/PrephixData/ct_datasets/PrephixNMDID/nifti/BONE_H-N-UXT_3X3/case-100065/segmentations/skull.nii.gz")

In [4]:
vol = Volume.from_nifti(vol_path, segmentation=True)
vol_cropped = vol.shrink(padding=5)
bodies = vol_cropped.split_bodies()

[b.volume for b in bodies]

[857257.2303771973,
 1518.0702209472656,
 11.90643310546875,
 23.8128662109375,
 735.2222442626953,
 5.953216552734375,
 1354.3567657470703,
 17.859649658203125]

In [5]:
# count nonzero
data = bodies[0].data
np.count_nonzero(data)

287998

In [7]:


# for body in bodies:
body = bodies[0]
body_shrunk = body.shrink(padding=5)
shrunk_downscaled = body_shrunk.downscale((3,3,1), segmentation=True)
segmented = shrunk_downscaled.cluster_seg(segmentation=True, n_clusters=9, expand=10)
segmented_upscaled = segmented.upscale((3,3,1), segmentation=True, smooth_seg=(3,3,3))

split_parts = body_shrunk.split_by_segmentation(segmented_upscaled.data)



/home/pelvisvr/mambaforge/envs/prephix/lib/python3.10/site-packages/sklearn/manifold/_spectral_embedding.py:285: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


In [39]:

part_iso_args = dict(
    # decimation=0.9,
    decimation=0,
    # decimation_points=None,
    decimation_points=4000,
    smooth=False,
    convert_to_LPS=True,
    taubin_smooth=True,
    # taubin_smooth_iter=10,
    taubin_smooth_pass_band=0.1
)


part_names = []
for i, part in enumerate(split_parts):
    part_poly = part.isosurface(**part_iso_args)
    # part_poly = part.isosurface(decimation=0, smooth_iter=0, smooth=False,)
    print(f"Part {i} has {len(part_poly.faces)} points, {part.volume} volume")
    part_path = f"part_{i}.stl"
    part_poly.save(part_path)
    part_names.append(part_path)

combined_parts = pv.PolyData()
for part_name in part_names:
    part_poly = pv.read(part_name)
    if len(part_poly.points) == 0:
        print(f"Skipping {part_name}")
        continue
    combined_parts = combined_parts.merge(part_poly)

# save as combined.stl
combined_parts.save("combined.stl")

Part 0 has 31992 points, 173729.7420501709 volume
Part 1 has 32004 points, 261634.93766784668 volume
Part 2 has 31988 points, 171172.83554077148 volume
Part 3 has 32000 points, 159126.50184631348 volume
Part 4 has 31984 points, 28319.451141357422 volume
Part 5 has 23264 points, 9224.509048461914 volume
Part 6 has 31936 points, 12576.169967651367 volume
Part 7 has 31984 points, 33766.644287109375 volume
Part 8 has 17984 points, 7706.438827514648 volume
